In [2]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import copy

In [16]:
patchSize = 128
imgWidth = 400
imgHeight = 300
pertubation = 32

maxWidth = imgWidth - patchSize
maxHeight = imgHeight - patchSize
widthOrigin = random.randint(0, maxWidth)
heightOrigin = random.randint(0, maxHeight)

topLeft = (widthOrigin, heightOrigin)
topRight = (widthOrigin + patchSize, heightOrigin)
bottomLeft = (widthOrigin, heightOrigin + patchSize)
bottomRight = (widthOrigin + patchSize, heightOrigin + patchSize)
corners = [topLeft, topRight, bottomLeft, bottomRight]

pertubratedCorners = []
for corner in corners:
    x = min(imgWidth, max(0, corner[0] + random.randint(-pertubation, pertubation)))
    y = min(imgHeight, max(0, corner[1] + random.randint(-pertubation, pertubation)))
    pertubratedCorner = (x, y)
    pertubratedCorners.append(pertubratedCorner)
corners = np.float32(corners)
pertubratedCorners = np.float32(pertubratedCorners)

M = cv2.getPerspectiveTransform(corners, pertubratedCorners)
print(corners)
print(pertubratedCorners)
print(M)
Minv = np.linalg.inv(M)

H4Pt = pertubratedCorners - corners
H4Pt = np.reshape(H4Pt, 8)

[[222.  36.]
 [350.  36.]
 [222. 164.]
 [350. 164.]]
[[202.  19.]
 [330.  39.]
 [225. 141.]
 [368. 192.]]
[[ 3.93902357e-01  6.08513876e-02  6.16988514e+01]
 [ 7.50722933e-02  6.67841029e-01 -2.64737684e+01]
 [-1.07662089e-03 -3.27858652e-04  1.00000000e+00]]


In [21]:
# input: corner A, predicted H4pt
# output: Homography

predH4Pt = copy.copy(H4Pt)

cornersA = np.reshape(corners, 8)
cornersB = cornersA + predH4Pt

b = np.zeros(8)
A = np.zeros((8, 8))
for i in range(4):
    b[2 * i] = -cornersB[2 * i + 1]
    b[2 * i + 1] = cornersB[2 * i]

    A[2 * i, 0:3] = np.zeros(3)
    A[2 * i + 1, 3:6] = np.zeros(3)
    A[2 * i, 3:6] = np.array([-cornersA[2 * i], -cornersA[2 * i + 1], -1])
    A[2 * i + 1, 0:3] = np.array([cornersA[2 * i], cornersA[2 * i + 1], 1])
    A[2 * i, 6:8] = np.array([cornersB[2 * i + 1] * cornersA[2 * i],
                              cornersB[2 * i + 1] * cornersA[2 * i + 1]])
    A[2 * i + 1, 6:8] = np.array([-cornersB[2 * i] * cornersA[2 * i],
                                  -cornersB[2 * i] * cornersA[2 * i + 1]])

Ainv = np.linalg.pinv(A)
h8 = np.matmul(Ainv, b)

H = np.concatenate((h8, np.ones(1)))
H = np.reshape(H, (3, 3))

print(H)
print(M)
print(H - M)


[[ 3.93902357e-01  6.08513876e-02  6.16988514e+01]
 [ 7.50722933e-02  6.67841029e-01 -2.64737684e+01]
 [-1.07662089e-03 -3.27858652e-04  1.00000000e+00]]
[[ 3.93902357e-01  6.08513876e-02  6.16988514e+01]
 [ 7.50722933e-02  6.67841029e-01 -2.64737684e+01]
 [-1.07662089e-03 -3.27858652e-04  1.00000000e+00]]
[[ 1.80905291e-12 -4.17129525e-12 -1.47160506e-10]
 [ 1.13395404e-13 -9.64450741e-13  1.16173737e-12]
 [ 4.82708491e-15 -1.52945148e-14  0.00000000e+00]]
